In [1]:
from metrics import *
from utils import *
from data_utils import *
from plotting import *
from features import *
import lstm
import pdb
import pandas as pd
import numpy as np
import keras
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential
import plotly
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *
from sklearn.preprocessing import StandardScaler, MinMaxScaler

init_notebook_mode(connected=True)

#DATAFILE = '../analyser/rawData/2018-01-15_ETHBTC_1m_6_mon_slice_last_50k.csv'
#DATAFILE = '../analyser/rawData/2018-01-15_ETHBTC_1m_6_mon_.csv'
DATAFILE = '../analyser/rawData/2018-01-15_ETHBTC_1h_4_mon_.csv'
#DATAFILE = '../analyser/rawData/2018-02-13_ETHBTC_5m_2_mon_.csv'
#DATAFILE = '../analyser/rawData/test-sample-mixed.csv'
#DATAFILE = '../analyser/rawData/test-sample-flat.csv'
#DATAFILE = '../analyser/rawData/test-sample-falling.csv'
#DATAFILE = '../analyser/rawData/2018-02-13_ETHBTC_1m_2_mon_.csv'
#DATAFILE = '../analyser/rawData/test-sample-short.csv'
#DATAFILE = 'test.csv'
COLUMN_NAME = 'close'

#DATAFILE = 'shampoo-data.csv'
#COLUMN_NAME = 'Sales'
#DATAFILE = 'sinwave.csv'
#COLUMN_NAME = 'sin'

/Users/vvtaranov/Envs/tading-bot-tensorflow/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6

/Users/vvtaranov/Envs/tading-bot-tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning:

Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.

Using TensorFlow backend.


In [2]:
# Load data
data = pd.read_csv(DATAFILE)
prices = data[COLUMN_NAME]
a = """
pricesLine = Scatter(
            x=prices.index,
            y=prices,
            name='prices'
            )
iplot([pricesLine], config={ "scrollZoom": True })
"""



In [3]:
# Calculate features
features = getFeatures(opn=data.open.values,
                            close=data.close.values,
                            high=data.high.values,
                            low=data.low.values,
                            volume=data.volume.values)
# Remove the last value since it doesn't have a next one, so it doesn't have a known output
features = features[:-1]
nanFeatures = np.isnan(features).any(axis=1)
# Remove the values with nans
xdata = features[~nanFeatures]

In [4]:
# Calculate outputs: log returns
# Drop the nan at the start to align inputs with outputs
diffPrices = np.log(prices).diff().dropna().values
ydata = diffPrices[~nanFeatures]
ydata = ydata.reshape(ydata.shape[0], 1)

In [5]:
# Split training set and test set
xtrain, xtest = splitDataset(0.9, 0.1, xdata)
ytrain, ytest = splitDataset(0.9, 0.1, ydata)

print('Training examples: {}'.format(xtrain.shape))
print('Test examples: {}'.format(xtest.shape))

Training examples: (2316, 110)
Test examples: (257, 110)


In [6]:
# Normalize things set
scaler = StandardScaler()
xtrain = scaler.fit_transform(xtrain)
xtest = scaler.transform(xtest)

In [7]:
# Reshape to proper input form
xtrain = xtrain.reshape(xtrain.shape[0], 1, xtrain.shape[1])
xtest = xtest.reshape(xtest.shape[0], 1, xtest.shape[1])

In [24]:
# Train the model
EPOCHS = 200
"""
model = Sequential()
model.add(LSTM(128,
    batch_input_shape=(1, xtrain.shape[1], xtrain.shape[2]),
    stateful=True
    ))
model.add(Dense(64))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(loss="mean_squared_error", optimizer="adam")
reduce_lr = keras.callbacks.ReduceLROnPlateau(
            monitor='loss',
            factor=0.9,
            patience=10,
            min_lr=0.000001,
            verbose=1)
print("Start training for {} epochs".format(EPOCHS))
"""
for i in range(EPOCHS):
    model.fit(
        xtrain,
        ytrain,
        epochs=1,
        batch_size=1,
        verbose=1,
        shuffle=False,
        callbacks=[reduce_lr]
    )
    model.reset_states()
    print("{}/{} epoch complete".format(i + 1, EPOCHS))


Epoch 1/1
2316/2316 [==============================] - 10s 4ms/step - loss: 1.0383e-04
1/200 epoch complete
Epoch 1/1
2316/2316 [==============================] - 10s 4ms/step - loss: 1.1595e-04
2/200 epoch complete
Epoch 1/1
2316/2316 [==============================] - 9s 4ms/step - loss: 1.0784e-04
3/200 epoch complete
Epoch 1/1
2316/2316 [==============================] - 10s 4ms/step - loss: 9.7735e-05
4/200 epoch complete
Epoch 1/1
2316/2316 [==============================] - 8s 4ms/step - loss: 1.0078e-04
5/200 epoch complete
Epoch 1/1
2316/2316 [==============================] - 9s 4ms/step - loss: 1.2672e-04
6/200 epoch complete
Epoch 1/1
2316/2316 [==============================] - 8s 4ms/step - loss: 1.0960e-04
7/200 epoch complete
Epoch 1/1
2316/2316 [==============================] - 8s 4ms/step - loss: 1.1021e-04
8/200 epoch complete
Epoch 1/1
2316/2316 [==============================] - 8s 4ms/step - loss: 1.0461e-04
9/200 epoch complete
Epoch 1/1
2316/2316 [=============

76/200 epoch complete
Epoch 1/1
2316/2316 [==============================] - 8s 4ms/step - loss: 9.9509e-05
77/200 epoch complete
Epoch 1/1
2316/2316 [==============================] - 8s 4ms/step - loss: 9.2192e-05
78/200 epoch complete
Epoch 1/1
2316/2316 [==============================] - 8s 4ms/step - loss: 9.4701e-05
79/200 epoch complete
Epoch 1/1
2316/2316 [==============================] - 8s 4ms/step - loss: 9.8450e-05
80/200 epoch complete
Epoch 1/1
2316/2316 [==============================] - 8s 4ms/step - loss: 9.7961e-05
81/200 epoch complete
Epoch 1/1
2316/2316 [==============================] - 8s 4ms/step - loss: 9.3090e-05
82/200 epoch complete
Epoch 1/1
2316/2316 [==============================] - 8s 4ms/step - loss: 1.0371e-04
83/200 epoch complete
Epoch 1/1
2316/2316 [==============================] - 8s 4ms/step - loss: 1.0194e-04
84/200 epoch complete
Epoch 1/1
2316/2316 [==============================] - 8s 4ms/step - loss: 9.6904e-05
85/200 epoch complete
Epoch 

2316/2316 [==============================] - 9s 4ms/step - loss: 9.1508e-05
152/200 epoch complete
Epoch 1/1
2316/2316 [==============================] - 10s 4ms/step - loss: 1.0293e-04
153/200 epoch complete
Epoch 1/1
2316/2316 [==============================] - 9s 4ms/step - loss: 9.4792e-05
154/200 epoch complete
Epoch 1/1
2316/2316 [==============================] - 9s 4ms/step - loss: 9.3307e-05
155/200 epoch complete
Epoch 1/1
2316/2316 [==============================] - 10s 4ms/step - loss: 9.6278e-05
156/200 epoch complete
Epoch 1/1
2316/2316 [==============================] - 9s 4ms/step - loss: 9.5933e-05
157/200 epoch complete
Epoch 1/1
2316/2316 [==============================] - 9s 4ms/step - loss: 8.6423e-05
158/200 epoch complete
Epoch 1/1
2316/2316 [==============================] - 10s 4ms/step - loss: 9.5491e-05
159/200 epoch complete
Epoch 1/1
2316/2316 [==============================] - 9s 4ms/step - loss: 9.0033e-05
160/200 epoch complete
Epoch 1/1
2316/2316 [=====

In [25]:
# Evaluate
# Preseed with train data
PRESEED_AMOUNT = 500
preseedX = xtrain[-PRESEED_AMOUNT:]
preseedX
for i in range(preseedX.shape[0]):
    model.predict(preseedX[i].reshape(1, preseedX.shape[1], preseedX.shape[2]))

# Predict the values from test
pred = np.array([[0]])
for i in range(xtest.shape[0]):
    pr = model.predict(xtest[i].reshape(1, xtest.shape[1], xtest.shape[2]))
    pred = np.concatenate((pred, pr))

pred = pred[1:]
model.reset_states()

In [26]:
# Plot as it is
predLine = Scatter(
            x=np.arange(pred.shape[0]),
            y=pred.flatten(),
            name='predictions'
            )
actualLine = Scatter(
            x=np.arange(ytest.shape[0]),
            y=ytest.flatten(),
            name='actual'
            )
iplot([predLine, actualLine], config={ "scrollZoom": True })



In [ ]:
# Check for errors
# Save the model for later
# Try predicting a few minutes ahead one-by-one
